In [40]:
import tensorflow as tf
import tensorflow.keras.backend as T
from tensorflow.keras.layers import Layer

class DCCA(Layer):
    '''CCA layer is used compute the CCA objective

    # Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.

    # Output shape
        Same shape as the input.

    # Arguments
        output_dim: output dimension, default 1, i.e., correlation coefficient
        use_all_singular_value: if use the top-k singular values
        cca_space_dim: the number of singular values, i.e., k

    '''

    def __init__(self, output_dim=1, use_all_singular_values=True, cca_space_dim=10, **kwargs):
        self.output_dim = output_dim
        self.cca_space_dim = cca_space_dim
        self.use_all_singular_values = use_all_singular_values
        super(DCCA, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        super(DCCA, self).build(input_shape)

    def call(self, x):
        r1 = tf.constant([1e-4])
        r2 = tf.constant([1e-4])
        eps = tf.constant([1e-12])
        o1 = o2 = tf.shape(x)[1] // 2

        H1 = T.transpose(x[:, 0:o1])
        H2 = T.transpose(x[:, o1:o1 + o2])

        one = tf.constant([1.0])
        m = tf.shape(H1)[1]
        m_float = tf.cast(m, 'float')

        # minus the mean value
        partition = tf.divide(one, m_float)
        H1bar = H1 - partition * tf.matmul(H1, tf.ones([m, m]))
        H2bar = H2 - partition * tf.matmul(H2, tf.ones([m, m]))

        # calculate the auto-covariance and cross-covariance
        partition2 = tf.divide(one, (m_float - 1))
        SigmaHat12 = partition2 * tf.matmul(H1bar, tf.transpose(H2bar))
        SigmaHat11 = partition2 * tf.matmul(H1bar, tf.transpose(H1bar)) + r1 * tf.eye(o1)
        SigmaHat22 = partition2 * tf.matmul(H2bar, tf.transpose(H2bar)) + r2 * tf.eye(o2)

        # calculate the root inverse of covariance matrices by using eigen decomposition
        D1, V1 = tf.linalg.eigh(SigmaHat11)
        D2, V2 = tf.linalg.eigh(SigmaHat22)

        # for stability
        D1_indices = tf.where(D1 > eps)
        D1_indices = tf.squeeze(D1_indices)
        V1 = tf.gather(V1, D1_indices)
        D1 = tf.gather(D1, D1_indices)

        D2_indices = tf.where(D2 > eps)
        D2_indices = tf.squeeze(D2_indices)
        V2 = tf.gather(V2, D2_indices)
        D2 = tf.gather(D2, D2_indices)

        pow_value = tf.constant([-0.5])
        SigmaHat11RootInv = tf.matmul(tf.matmul(V1, tf.linalg.diag(tf.pow(D1, pow_value))), tf.transpose(V1))
        SigmaHat22RootInv = tf.matmul(tf.matmul(V2, tf.linalg.diag(tf.pow(D2, pow_value))), tf.transpose(V2))

        Tval = tf.matmul(tf.matmul(SigmaHat11RootInv, SigmaHat12), SigmaHat22RootInv)

        if self.use_all_singular_values:
            # all singular values are used to calculate the correlation
            corr = tf.linalg.trace(T.sqrt(tf.matmul(tf.transpose(Tval), Tval)))
        else:
            # just the top outdim_size singular values are used
            TT = tf.matmul(tf.transpose(Tval), Tval)
            U, V = tf.raw_ops.SelfAdjointEigV2(input=TT)
            U_sort, _ = tf.nn.top_k(U, self.cca_space_dim)
            corr = T.sum(T.sqrt(U_sort))

        corr = tf.fill([tf.shape(x)[0], self.output_dim], corr)

        return -corr

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        config = {
            'output_dim': self.output_dim,
            'cca_dim': self.cca_dim,
            'use_all_singular_values': self.use_all_singular_values,
        }
        base_config = super(CCA, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [84]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, concatenate, Dropout
import tensorflow.keras.backend as K


def constant_loss(y_true, y_pred):
    return y_pred

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
X = df[['sepal_length', 'sepal_length', 'sepal_width']][:50]
Y = df[['petal_length', 'petal_length', 'petal_width']][:50]

# size of the input for view 1 and view 2
input_shape1 = 3
input_shape2 = 3

# network settings
epoch_num = 10
batch_size = 100

#load data

train_set_x1 = df[['sepal_length', 'sepal_length', 'sepal_width']][:100]
valid_set_x1 = df[['sepal_length', 'sepal_length', 'sepal_width']][:50]

train_set_x2 = df[['petal_length', 'petal_length', 'petal_width']][:100]
valid_set_x2 = df[['petal_length', 'petal_length', 'petal_width']][:50]

input1 = Input(shape=(input_shape1, ), name='input1')
input2 = Input(shape=(input_shape1, ), name='input2')

expert_index = 0
activation_model = 'sigmoid'
dense1_1 = Dense(128, activation=activation_model, name='view_1_1')(input1)
dense1_2 = Dense(128, activation=activation_model, name='view_1_2')(dense1_1)
dense1_3 = Dense(128, activation=activation_model,  name='view_1_3')(dense1_2)
output1 = Dense(2, activation='linear', name='view_1_4')(dense1_3)

dense2_1 = Dense(128, activation=activation_model,  name='view_2_1')(input2)
dense2_2 = Dense(128, activation=activation_model,  name='view_2_2')(dense2_1)
dense2_3 = Dense(128, activation=activation_model, name='view_2_3')(dense2_2)
output2 = Dense(2, activation='linear', name='view_2_4')(dense2_3)

shared_layer = concatenate([output1, output2], name='shared_layer')

cca_layer = CCA(1, name='cca_layer')(shared_layer)

model = Model(inputs=[input1, input2], outputs=cca_layer)
model.compile(optimizer='rmsprop', loss=constant_loss, metrics=[mean_pred])
model.fit([train_set_x1, train_set_x2], np.zeros(len(train_set_x1)),
          batch_size=batch_size, epochs=epoch_num, shuffle=True, verbose=1,
          validation_data=([valid_set_x1, valid_set_x2], np.zeros(len(valid_set_x1))))


Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: -0.0915 - mean_pred: -0.0915 - val_loss: -0.0549 - val_mean_pred: -0.0549
Epoch 2/10
1/1 [==============================] - 0s 73ms/step - loss: -0.9318 - mean_pred: -0.9318 - val_loss: -0.0943 - val_mean_pred: -0.0943
Epoch 3/10
1/1 [==============================] - 0s 67ms/step - loss: -1.0361 - mean_pred: -1.0361 - val_loss: -0.1126 - val_mean_pred: -0.1126
Epoch 4/10
1/1 [==============================] - 0s 77ms/step - loss: -1.1592 - mean_pred: -1.1592 - val_loss: -0.1244 - val_mean_pred: -0.1244
Epoch 5/10
1/1 [==============================] - 0s 79ms/step - loss: -1.1980 - mean_pred: -1.1980 - val_loss: -0.1292 - val_mean_pred: -0.1292
Epoch 6/10
1/1 [==============================] - 0s 65ms/step - loss: -1.2184 - mean_pred: -1.2184 - val_loss: -0.1342 - val_mean_pred: -0.1342
Epoch 7/10
1/1 [==============================] - 0s 60ms/step - loss: -1.2313 - mean_pred: -1.2313 - val_loss: -0.1383 - val_mean_p

In [85]:
train_set_x1.shape

(100, 3)

In [88]:
# evaluation for view_1
new_data    = []
expert_data = []
expert_result = []
current_dcca = Model(model.input, model.get_layer(name='shared_layer').output)


pred_out = current_dcca.predict([X, Y])


2/2 [==============================] - 0s 7ms/step


In [79]:
model.get_layer(name='shared_layer').output

<KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'shared_layer')>

In [80]:
pred_out.shape

(150, 4)

In [105]:
print(np.corrcoef(pred_out[:,0], pred_out[:,2]))

[[1.        0.1958496]
 [0.1958496 1.       ]]


In [106]:
print(np.corrcoef(pred_out[:,1], pred_out[:,3]))

[[1.         0.15546917]
 [0.15546917 1.        ]]


In [93]:
ca = CCA(n_components=2)
ca.fit(train_set_x1, train_set_x2)
X_c, Y_c = ca.transform(X, Y)


In [102]:
print(np.corrcoef(X_c[:,0], Y_c[:,0]))

[[1.         0.11540336]
 [0.11540336 1.        ]]


In [104]:
print(np.corrcoef(X_c[:,1], Y_c[:,1]))

[[1.         0.12859254]
 [0.12859254 1.        ]]


In [96]:
print(np.corrcoef(X_c[:,1], Y_c[:,1]))

[[1.         0.12859254]
 [0.12859254 1.        ]]
